In [1]:
import numpy as np, prjlib, plottools as pl, healpy as hp, pickle, curvedsky, delens_tools, quad_func, misctools
from matplotlib.pyplot import *
from IPython.display import clear_output

In [2]:
def prep_window(wtype,fmaskla,fmasksa,nsidesa=512):
    if wtype in ['i','W']:
        wfac = None
        #Wla = 1.
    else:
        wfac = {}
        __, __, wfac['fsky'], wfac['w1'], wfac['w2'], wfac['w4'] = prjlib.window(fmaskla)
        #Mla0 = hp.pixelfunc.ud_grade(Mla,nsidesa)
        #Wla = hp.pixelfunc.ud_grade(wla,nsidesa)
    Wsa, __, __, __, __, __ = prjlib.window(fmasksa)
    return wfac, Wsa

In [3]:
def prep_qrec(wtype,p,f,r,ow=False,vb=False):
    if wtype in ['W','i']:
        ocl = prjlib.loadocl(f.cmb.scl['o'],lTmin=p.lTmin,lTmax=p.lTmax)
        ifl = r.lcl[0:3,:]
    if wtype == 'w':
        ocl, ifl = prjlib.quad_filter(f.cmb.scl,p.lmax,r.lcl,sinp=wtype,lTmin=p.lTmin,lTmax=p.lTmax)
    quad_func.quad.cinvfilter(p.quad,ocl=ifl)
    quad_func.quad.al(p.quad,r.lcl,ocl,overwrite=ow,verbose=vb)
    quad_func.quad.qrec(p.quad,p.snmin,p.snmax,f.cmb.alms[wtype],r.lcl,overwrite=ow,verbose=vb)

In [4]:
def plot_al():
    pl.plot_1dstyle(fsize=[7,4],xmin=20,xmax=3000,xlog=True,ylog=True,ymin=5e-9,ymax=4e-6,grid=True)
    Al = {}
    for q in ['TT','TE','EE','EB']:
        Al[q] = np.loadtxt(p.quad.f[q].al,unpack=True)[1]
        plot(Al[q],label=q)
    legend()

In [17]:
def delens(wtype,p,psa,fco,fid,Wsa,r,ep=1e-30,olmax=1024,qlist=['TT','TE','PO','TP'],rmmean=False,wfac=None):
    p.quad.qlist.extend(['PO','TP'])
    wlk = prjlib.Wiener_Lensing(p.quad.f,r.kk,p.dlmin,p.dlmax,kL=r.kL,qlist=p.quad.qlist)
    cl = {}
    LT = {}
    for q in qlist:  cl[q] = np.zeros((p.snmax-p.snmin+1,3,olmax+1))
    for i in p.rlz:
        misctools.progress(i,p.rlz)
        wElm = pickle.load(open(fco.cmb.alms['w']['E'][i],"rb"))[:p.dlmax+1,:p.dlmax+1]
        Balm = pickle.load(open(fid.cmb.alms['o']['B'][i],"rb"))[:olmax+1,:olmax+1]
        __, wBlm = prjlib.multiplywindow_spin(Wsa,psa.npix,psa.nside,olmax,0*Balm,Balm)
        klm = delens_tools.prep_klms(p.quad.f,i,p.dlmax,wfac=wfac,remove_mean=rmmean,qlist=p.quad.qlist)
        LT[i] = {}
        for q in qlist:
            print(q)
            if q=='TT':
                wplm = 0.*klm[q]
                for n0, n1 in [(32,64),(64,128),(128,256),(256,512),(512,1024)]:
                    l0, l1 = 2*n0+1, 2*n1
                    skap, nkap, wphi = pickle.load(open('trash/tmp_w_'+str(n1)+'.pkl',"rb"))
                    #wphi = nkap/(skap+1e-30)
                    wlm = klm[q]*wlk[q][:,None]
                    wlm[:l0,:] = 0.
                    tmp = prjlib.multiplywindow(wphi,12*n1**2,n1,l1,wlm[:l1+1,:l1+1])
                    wplm[l0:l1+1,:l1+1] = tmp[l0:l1+1,:l1+1]
                    if n0==32:  wplm[:l0,:] = wlm[:l0,:]
            else:
                wplm = klm[q]*wlk[q][:,None]
            dalm = curvedsky.delens.lensingb(olmax,p.dlmin,p.dlmax,p.dlmin,p.dlmax,wElm,wplm)
            __, dalm = prjlib.multiplywindow_spin(Wsa,psa.npix,psa.nside,olmax,0*dalm,dalm)
            ii = i - p.snmin
            cl[q][ii,0,:] = curvedsky.utils.alm2cl(olmax,wBlm) + ep
            cl[q][ii,1,:] = curvedsky.utils.alm2cl(olmax,dalm) + ep
            cl[q][ii,2,:] = curvedsky.utils.alm2cl(olmax,dalm,wBlm) + ep
            LT[i][q] = dalm
    pl.plot_1dstyle(fsize=[8,6],xmin=3,xmax=200,ymin=.0,ymax=.6,grid=True,ylab=r'$C_L^{\rm LT}/C_L^{\rm Lens}$')
    for q in qlist:
        plot(np.mean(cl[q][:,2,:]**2/(cl[q][:,0,:]*cl[q][:,1,:]),axis=0),label=q)
    legend()
    savefig('fig_delens_'+wtype+'.png')
    clf()
    return cl, LT

In [18]:
psa, fsa, __ = prjlib.analysis_init(t='sa',freq='coadd')
fid = prjlib.filename_init(t='id')
fco = prjlib.filename_init(t='co',freq='coadd')
bb, LT = {}, {}
#for wtype in ['i','W','w']:
for wtype in ['w']:
    if wtype == 'i':  exttag, rmmean = '_iso', False
    if wtype == 'W':  exttag, rmmean = '_wdiag', True
    if wtype == 'w':  exttag, rmmean = '', True
    p, f, r = prjlib.analysis_init(t='la',freq='coadd',rlmin=500,rlmax=3000,snmin=1,snmax=1,exttag=exttag,qlist=['TT','TE','EE','EB'])
    wfac, Wsa = prep_window(wtype,f.cmb.mask,fsa.cmb.mask,nsidesa=psa.nside)
    #prep_qrec(wtype,p,f,r,ow=False,vb=True)
    bb[wtype], LT[wtype] = delens(wtype,p,psa,fco,fid,Wsa,r,rmmean=rmmean,wfac=wfac,qlist=['TT'])

Current progress : 0.0 %
TT


<Figure size 576x432 with 0 Axes>

In [ ]:
olmax=1024
wtype = 'w'
cbb = np.zeros((p.snmax-p.snmin+1,olmax+1))
vec = np.zeros((p.snmax-p.snmin+1,3,olmax+1))
mat = np.zeros((p.snmax-p.snmin+1,3,3,olmax+1))
for i in range(p.snmin,p.snmax+1):
    ii = i - p.snmin
    Balm = pickle.load(open(fid.cmb.alms['o']['B'][i],"rb"))[0:olmax+1,0:olmax+1]
    __, wBlm = prjlib.multiplywindow_spin(Wsa,psa.npix,psa.nside,olmax,0*Balm,Balm)
    cbb[ii,:] = curvedsky.utils.alm2cl(olmax,wBlm)
    for qi, q0 in enumerate(['TT','TE','PO']):
        vec[ii,qi,:] = curvedsky.utils.alm2cl(olmax,LT[wtype][i][q0],wBlm)
        for qj, q1 in enumerate(['TT','TE','PO']):
            mat[ii,qi,qj,:] = curvedsky.utils.alm2cl(olmax,LT[wtype][i][q0],LT[wtype][i][q1])

In [ ]:
ylim(0.,0.6)
xlim(2,200)
mvec = np.mean(vec,axis=0)
mmat = np.mean(mat,axis=0)
rho = np.zeros(olmax+1)
bb = np.mean(cbb,axis=0)
for l in range(2,olmax):
    rho[l] = np.dot(mvec[:,l],np.dot(np.linalg.inv(mmat[:,:,l]),mvec[:,l]))
plot(rho/bb)
savefig('fig_delens_wopt.png')